## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
for col in application_df:
  print(f"{col}: {application_df[col].nunique()}")

APPLICATION_TYPE: 17
AFFILIATION: 6
CLASSIFICATION: 71
USE_CASE: 5
ORGANIZATION: 4
STATUS: 2
INCOME_AMT: 9
SPECIAL_CONSIDERATIONS: 2
ASK_AMT: 8747
IS_SUCCESSFUL: 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T2','T7','T8','T9','T9','T10','T12','T13','T14','T15','T17','T29']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2263
T4        1542
T6        1216
T5        1173
T19       1065
T25          3
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count[classification_count != 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df['CLASSIFICATION'].unique())
to_keep = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']
for each in to_keep:
  classifications_to_replace.remove(each)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_dummies = pd.get_dummies(application_df[application_cat])
application_val = application_df.dtypes[application_df.dtypes == "int64"].index.tolist()
application_preprocessed = pd.concat([application_dummies, application_df[application_val]], axis = 1)
application_preprocessed.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T25,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,5000,1
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,108590,1
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,1,5000,0
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,6692,1
4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,142590,1


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_preprocessed['IS_SUCCESSFUL'].values
X = application_preprocessed.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

C:\Users\zohai\AppData\Local\Temp\ipykernel_10232\185012117.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_preprocessed.drop(['IS_SUCCESSFUL'],1).values


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 378       
                                                                 
 dense_1 (Dense)             (None, 4)                 40        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 423 (1.65 KB)
Trainable params: 423 (1.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6462 - accuracy: 0.6440
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5857 - accuracy: 0.7177
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5737 - accuracy: 0.7224
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5695 - accuracy: 0.7260
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5665 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5648 - accuracy: 0.7292
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5630 - accuracy: 0.7288
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5620 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5610 - accuracy: 0.7288
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5604 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5599 - accuracy: 0.7298 - 626ms/epoch - 2ms/step
Loss: 0.5598781704902649, Accuracy: 0.7297959327697754


In [16]:
#Optimize model with additional hidden layer
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
hidden_nodes_layer1_2 =  7
hidden_nodes_layer2_2 = 4
hidden_nodes_layer3_2 = 4

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 7)                 294       
                                                                 
 dense_4 (Dense)             (None, 4)                 32        
                                                                 
 dense_5 (Dense)             (None, 4)                 20        
                                                                 
 dense_6 (Dense)             (None, 1)                 5         
                                                                 
Total params: 351 (1.37 KB)
Trainable params: 351 (1.37 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
#compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
#train the model (make it hit the gym)
fit_model = nn_2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6610 - accuracy: 0.6359
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5770 - accuracy: 0.7205
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5642 - accuracy: 0.7273
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5601 - accuracy: 0.7274
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5581 - accuracy: 0.7283
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5572 - accuracy: 0.7274
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7286
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7292
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5550 - accuracy: 0.7292
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7294 - 493ms/epoch - 2ms/step
Loss: 0.5551134943962097, Accuracy: 0.7294460535049438


In [20]:
application_preprocessed.corr()['IS_SUCCESSFUL'].sort_values()

AFFILIATION_CompanySponsored   -0.369379
ORGANIZATION_Association       -0.179561
CLASSIFICATION_C2100           -0.147895
APPLICATION_TYPE_T19           -0.123303
APPLICATION_TYPE_T4            -0.098692
CLASSIFICATION_C1200           -0.056621
INCOME_AMT_0                   -0.052509
USE_CASE_CommunityServ         -0.039122
INCOME_AMT_50M+                -0.024838
CLASSIFICATION_C3000           -0.023182
INCOME_AMT_1M-5M               -0.022535
INCOME_AMT_10M-50M             -0.021575
USE_CASE_Heathcare             -0.013222
USE_CASE_ProductDev            -0.011841
ASK_AMT                        -0.010861
INCOME_AMT_5M-10M              -0.009171
SPECIAL_CONSIDERATIONS_N       -0.005469
STATUS                         -0.001636
APPLICATION_TYPE_T3            -0.000953
CLASSIFICATION_C2000           -0.000434
APPLICATION_TYPE_T25            0.002517
AFFILIATION_Family/Parent       0.002608
AFFILIATION_Regional            0.003238
AFFILIATION_National            0.004581
SPECIAL_CONSIDER

In [21]:
application_preprocessed['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [22]:
application_preprocessed_2 = application_preprocessed.drop(['STATUS'], axis = 1)
application_preprocessed_2.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T25,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT,IS_SUCCESSFUL
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,5000,1
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,108590,1
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,5000,0
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,6692,1
4,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,142590,1


In [23]:
# Split our preprocessed data into our features and target arrays
y_2 = application_preprocessed_2['IS_SUCCESSFUL'].values
X_2 = application_preprocessed_2.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=1)

C:\Users\zohai\AppData\Local\Temp\ipykernel_10232\1009175802.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_2 = application_preprocessed_2.drop(['IS_SUCCESSFUL'],1).values


In [24]:
# Create a StandardScaler instances
scaler_2 = StandardScaler()

# Fit the StandardScaler
X_scaler_2 = scaler.fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler_2.transform(X_train_2)
X_test_scaled_2 = X_scaler_2.transform(X_test_2)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
hidden_nodes_layer1_2 =  7
hidden_nodes_layer2_2 = 4
hidden_nodes_layer3_2 = 4

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features-1, activation="relu"))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 7)                 287       
                                                                 
 dense_8 (Dense)             (None, 4)                 32        
                                                                 
 dense_9 (Dense)             (None, 4)                 20        
                                                                 
 dense_10 (Dense)            (None, 1)                 5         
                                                                 
Total params: 344 (1.34 KB)
Trainable params: 344 (1.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn_3.fit(X_train_scaled_2,y_train_2,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6197 - accuracy: 0.6906
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5691 - accuracy: 0.7258
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5633 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5612 - accuracy: 0.7260
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5599 - accuracy: 0.7273
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7269
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5581 - accuracy: 0.7275
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7283
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7282
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accura

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled_2,y_test_2,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7263 - 531ms/epoch - 2ms/step
Loss: 0.5567384958267212, Accuracy: 0.7262973785400391


In [29]:
# Export our model to HDF5 file
nn_3.save('AlphabetSoupCharity_Optimization.h5', save_format = 'h5')

c:\Users\zohai\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1)
clf.fit(X_train_2, y_train_2)
clf.score(X_test_2, y_test_2)

0.7096209912536443